## LDO Analysis for C/ID

In [99]:
import numpy as np

In [100]:
## Design requirements
kT = 1.38e-23 * 300 # Boltzmann constant times temperature in Kelvin
PSRR_dB = -60
V_ctrl = 0.9
V_dd = 1.8
V_out = 1.2
f_bw = 1e6
I_load = 1e-3
PM_target = 60 # degrees
Noise_target = 250e-6 # Integrated noise target

In [101]:
## Determining Gain requirements of the Amplifier in feedback

Amp_gain_target = (10**(-1*PSRR_dB/20)) * (V_out / V_ctrl)
Amp_gain_target_dB = 20 * np.log10(Amp_gain_target)
print("Target Gain of the Amplifier in feedback: ", Amp_gain_target)
print("Target Gain of the Amplifier in feedback (dB): ", Amp_gain_target_dB)

Target Gain of the Amplifier in feedback:  1333.3333333333333
Target Gain of the Amplifier in feedback (dB):  62.498774732166


In [102]:
## Bandwidth estimation

C_L = 1.5e-12 # Load capacitance - starting value can change later if noise requirements are not met

## Assuming dominant pole comes from the main circuit
R_eff = 1/(f_bw * 2 * np.pi * C_L)
print("Effective Resistance seen by the load capacitance: ", R_eff)

I_total = (V_out / R_eff) + I_load
print("Total Current flowing through the lPMOS: ", I_total*1e3, "mA")

print("Overhead of the LDO: ", (I_total - I_load)*1e3, "mA")
print("overhead percentage of the LDO: ", ((I_total - I_load) / I_total) * 100, "%")

Effective Resistance seen by the load capacitance:  106103.29539459689
Total Current flowing through the lPMOS:  1.0113097335529233 mA
Overhead of the LDO:  0.011309733552923194 mA
overhead percentage of the LDO:  1.1183253930712158 %


In [103]:
## Amplifier bandwidth estimation
factor = np.tan(PM_target * np.pi / 180) # Converting PM to radians
factor = np.ceil(factor)
print("Bandwidth factor due to PM requirements: ", factor,"x")
Amp_bw = f_bw * factor # Assuming 10 times the bandwidth of the system

print("Target Bandwidth of the Amplifier: ", Amp_bw*1e-6, "MHz")

GBW_Amp = Amp_gain_target * Amp_bw
print("Gain-Bandwidth Product of the Amplifier: ", GBW_Amp*1e-9, "GHz")

Bandwidth factor due to PM requirements:  2.0 x
Target Bandwidth of the Amplifier:  2.0 MHz
Gain-Bandwidth Product of the Amplifier:  2.6666666666666665 GHz


In [104]:
## Noise estimation
Noise_bandwidth = 5*f_bw # Assuming a 2nd order system
PSD_target = Noise_target**2 / (f_bw) # Assuming uniform distribution of noise over the bandwidth
print("Power Spectral Density target: ", PSD_target, "V^2/Hz")

## Assume equal noise contribution from amp and non-amp
V_noise_amp = Noise_target / np.sqrt(2)
V_noise_LDO = Noise_target / np.sqrt(2)

print("Target noise contribution from Amp: ", V_noise_amp*1e6, "uV")


V_noise_Reff = np.sqrt((4*kT*R_eff)*Noise_bandwidth)
print("R_eff noise contribution: ", V_noise_Reff*1e6, "uV")

V_noise_PMOS_target = np.sqrt((V_noise_LDO**2) - (V_noise_Reff**2))
print("PMOS noise target: ", V_noise_PMOS_target*1e6, "uV")
PSD_PMOS_target = V_noise_PMOS_target**2 / Noise_bandwidth
gm_PMOS_target = PSD_PMOS_target/(4*kT*(R_eff**2))
print("Target gm of the pass transistor: ", gm_PMOS_target*1e3, "mS")

gm_ID_target = gm_PMOS_target / (I_total)
print("kgm target of the pass transistor: ", gm_ID_target)

Power Spectral Density target:  6.25e-14 V^2/Hz
Target noise contribution from Amp:  176.7766952966369 uV
R_eff noise contribution:  93.73021315815207 uV
PMOS noise target:  149.8821108115554 uV
Target gm of the pass transistor:  0.024099693510322186 mS
kgm target of the pass transistor:  0.023830180518143916


In [105]:
## Amplifier requirements
print("\n\nSummary of the Amplifier requirements:")
print("Gain: ", Amp_gain_target, " (", Amp_gain_target_dB, "dB )")
print("Output referred noise: ", V_noise_amp*1e6, "uV")
print("Bandwidth: ", Amp_bw*1e-6, "MHz")



Summary of the Amplifier requirements:
Gain:  1333.3333333333333  ( 62.498774732166 dB )
Output referred noise:  176.7766952966369 uV
Bandwidth:  2.0 MHz
